# Intake and Pelican FS Example

This is an example notebook that shows how to use `Intake-ESM` and `PelicanFS` together to access data that exists on a Pelican origin.  It will not actually analyze the data significantly as the exercise is to load in the data. Once that's done, everything else should follow naturally.

This is based off of the NCAR notebook [here](https://github.com/NCAR/cesm2-le-aws/blob/main/notebooks/kay_et_al_lens2.ipynb)

## Installation

Ensure that `PelicanFS` and `FSSpec` are both installed in the environment.

## Imports

In [1]:
%matplotlib inline
import warnings

warnings.filterwarnings("ignore")

import intake
import xarray as xr
from distributed import LocalCluster, Client
from ncar_jobqueue import NCARCluster

## Spin up a Cluster

This is just leftover from the original example, not a requirement for working with PelicanFS

In [2]:
# If not using NCAR HPC, use the LocalCluster
#cluster = LocalCluster()
cluster = NCARCluster()
cluster.scale(40)

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 40
Total threads: 120,Total memory: 160.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53605,Workers: 40
Dashboard: http://127.0.0.1:8787/status,Total threads: 120
Started: Just now,Total memory: 160.00 GiB
Comm: tcp://127.0.0.1:53616,Total threads: 3
Dashboard: http://127.0.0.1:53618/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:53608,


## Use the `Intake-ESM` Catalog to Access the Data

This catalog can exist in an origin, but since it's assumed objects in origins are immutable and it make require some testing to ensure the catalog is correct, it's recommended that you keep the catalog and csv files somewhere else until you are sure of their correctness. The catalog this example is using exists in the `resources` folder.


### Intake Format for PelicanFS

To have the catalog works with `PelicanFS`, give the namespace path and the appropriate protocol prefix. This is what a path the `.csv` file the `Intake` catalog uses looks like:

```
osdf:///chtc/PUBLIC/eturetsky/ncar-subset/ncar/monthly/cesm2LE-historical-cmip6-FLNS.zarr
```

Where `osdf://` is the protocol that indicates we are using the `osg` federation within. And `chtc/PUBLIC/eturetsky/ncar-subset/ncar/monthly/cesm2LE-historical-cmip6-FLNS.zarr` is the file's location within the `chtc/PUBLIC` namespace.

In [3]:
catalog = intake.open_esm_datastore(
        'file://resources/pelican-test-intake.json'
)
catalog

,unique
variable,4
long_name,4
component,1
experiment,1
forcing_variant,1
frequency,1
vertical_levels,1
spatial_domain,1
units,1
start_time,1


In [4]:
catalog_subset = catalog.search(variable='FLUT', frequency='monthly')
catalog_subset

catalog_subset2 = catalog.search(variable='FLNS', frequency='monthly')

In [5]:
catalog_subset.df

,variable,long_name,component,experiment,forcing_variant,frequency,vertical_levels,spatial_domain,units,start_time,end_time,path
0,FLUT,upwelling longwave flux at top of model,atm,historical,cmip6,monthly,1.0,global,W/m2,1850-01-16 12:00:00,2014-12-16 12:00:00,osdf:///chtc/PUBLIC/eturetsky/ncar-subset/ncar...


In [6]:
dsets = catalog_subset.to_dataset_dict()
dsets2 = catalog_subset2.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


In [7]:
dsets.keys()

dict_keys(['atm.historical.monthly.cmip6'])

In [8]:
dsets2.keys()

dict_keys(['atm.historical.monthly.cmip6'])

In [9]:
historical_cmip6 = dsets['atm.historical.monthly.cmip6']
other_cmip6 = dsets2['atm.historical.monthly.cmip6']

In [10]:
merge_ds = xr.concat([historical_cmip6, other_cmip6], dim = 'time')
merge_ds

<xarray.Dataset> Size: 88GB
Dimensions:    (member_id: 50, time: 3960, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) <U12 2kB 'r10i1181p1f1' ... 'r9i1301p1f1'
  * time       (time) object 32kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, nbnd) object 63kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    FLUT       (member_id, time, lat, lon) float32 44GB dask.array<chunksize=(1, 600, 192, 288), meta=np.ndarray>
    FLNS       (member_id, time, lat, lon) float32 44GB dask.array<chunksize=(1, 2580, 192, 288), meta=np.ndarray>
Attributes: (12/22)
    Conventions:                       CF-1.0
    NCO:                               netCDF Operators version 4.9.4 (Homepa...
    logname:                           sunseon
    model_doi_url:                     https://doi.org/10.5065/D67H1H0V
    source:                            CAM
    time_period_freq:                  month_1
    ...                                ...
    intake_esm_attrs:units:            W/m2
    intake_esm_attrs:start_time:       1850-01-16 12:00:00
    intake_esm_attrs:end_time:         2014-12-16 12:00:00
    intake_esm_attrs:path:             osdf:///chtc/PUBLIC/eturetsky/ncar-sub...
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_dataset_key:            atm.historical.monthly.cmip6